In [ ]:
import os
import boto3
import time
import numpy as np
import sagemaker
import torchvision
from matplotlib import pyplot as plt
from sagemaker.pytorch import PyTorch

from src.utils.utils import load_history, plot_history

sess = boto3.Session()
sm   = sess.client('sagemaker')
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
# role = 'coeuraj-ds'

bucket_name    = sagemaker_session.default_bucket()
jobs_folder    = 'jobs'
dataset_folder = 'datasets'

In [ ]:
# Parameters
backend = 'smddp' # 'gloo'
instance_type = 'local_gpu' #'ml.p4d.24xlarge', # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge'
instance_count = 1

In [ ]:
cifar10_dataset = torchvision.datasets.CIFAR10('cifar10-dataset',
                                               train=True,
                                               download=True)

In [ ]:
datasets = sagemaker_session.upload_data(path='cifar10-dataset',
                                         key_prefix=f'{dataset_folder}/cifar10-dataset')

In [ ]:
job_name   = f'pytorch-smddp-dist-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f's3://{bucket_name}/{jobs_folder}'

hyperparameters = {'epochs'         : 15,
                   'seed'           : 32,
                   'batch_size'     : 32,
                   # 'scheduler'      : None,
                   'optimizer'      : 'sgd',
                   'momentum'       : 0.9,
                   'lr'             : 0.001,
                   'criterion'      : 'cross_entropy',
                   # 'metric'         : None,
                   # 'model_dir'      : 'model_output',
                   'custom_function': True, # If True update custom_pre_process_function in src/utils/functions.py
                   'backend'        : backend
                   }

In [ ]:
distribution = { "smdistributed": {
                                    "dataparallel": { "enabled": True }
                                  }
                }

In [ ]:
estimator = PyTorch(entry_point          = 'main.py',
                    source_dir           = '.',
                    output_path          = output_path + '/',
                    code_location        = output_path,
                    role                 = role,
                    instance_count       = instance_count,
                    instance_type        = instance_type,
                    framework_version    = '1.11.0',
                    py_version           = 'py38',
                    distribution         = distribution,
                    hyperparameters      = hyperparameters)

In [ ]:
estimator.fit({'train': datasets},
              job_name=job_name,
              wait=True)

In [ ]:
history = load_history(os.environ["SM_MODEL_DIR"])

In [ ]:
# my_history = {
#     'train_loss': [2.1, 2.0, 1.9, 1.8, 1.7, 1.6],
#     'val_loss': [2.4, 2.3, 2.1, 1.9, 1.8, 1.7],
#     'epochs': [*range(1, 7)],
#     'train_metric': [0.5, 0.8, 0.85, 0.9, 0.98, 0.99],
#     'val_metric': [0.5, 0.6, 0.65, 0.7, 0.68, 0.87],
#     'metric_type': 'Accuracy'
# }

In [ ]:
plot_history(history)